In [ ]:
## DATA DOWNLOAD ##

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# List of stock symbols
stock_list = ["ioc"]#, "hindpetro", "bpcl", "ongc","nmdc","irfc"]

# Compute the date range
end_date = datetime.today() - timedelta(days=0)
start_date = datetime.today() - timedelta(days=200)

# Format dates as strings (yyyy-mm-dd)
start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

# Download data
for stock in stock_list:
    ticker = f"{stock}.NS"
    data = yf.download(ticker, start=start_str, end=end_str, interval="1d")
    data.to_excel(f"testing/{stock}20u0d.xlsx")


C:\Users\Ronu\AppData\Local\Temp\ipykernel_18180\3632997598.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_str, end=end_str, interval="1d")
[*********************100%***********************]  1 of 1 completed


In [ ]:
## CHECKING HTE DATA LEN##

In [37]:
import pandas as pd
for i in list:
    df=pd.read_excel(f"training\\{i}940d.xlsx")
    print(len(df))

902
902
902
902
902


In [33]:
## DELETING THE RECENT 40 DAYS DATA, WILL BE USED FOR TESTING ##

In [35]:
import os

# List of base filenames (without extension and path)
companies = ["irfc", "ioc", "hindpetro", "bpcl", "rvnl"]

for i in companies:
    file_path = f"training\\{i}940d.xlsx"
    
    try:
        # Read Excel file
        df = pd.read_excel(file_path)
        
        # Drop the last 40 rows
        df_trimmed = df[:-40]  # or df.iloc[:-40]

        # Save back to the same file (or a new one if you want to keep the original)
        df_trimmed.to_excel(file_path, index=False)
        print(f"{file_path}: trimmed and saved.")
    
    except Exception as e:
        print(f"Error with {file_path}: {e}")


training\irfc940d.xlsx: trimmed and saved.
training\ioc940d.xlsx: trimmed and saved.
training\hindpetro940d.xlsx: trimmed and saved.
training\bpcl940d.xlsx: trimmed and saved.
training\rvnl940d.xlsx: trimmed and saved.


In [41]:
import pandas as pd

file_paths = [r'training\irfc900d.xlsx', r'training\ioc900d.xlsx', r'training\hindpetro900d.xlsx']
for file in file_paths:
    df = pd.read_excel(file)
    print(f"Columns in {file}: {df.columns.tolist()}")

Columns in training\irfc900d.xlsx: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Columns in training\ioc900d.xlsx: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
Columns in training\hindpetro900d.xlsx: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']


In [63]:
import pandas as pd
from datetime import datetime, timedelta
import logging
import os

# Set up logging for outliers
logging.basicConfig(filename='outliers.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s')

# Define file paths
input_file = r'training\hindpetro900d.xlsx'
output_file = r'training\csv\hindpetro900d.csv'

# Check if input file exists
if not os.path.exists(input_file):
    raise FileNotFoundError(f"Input file '{input_file}' not found. Please verify the file path.")

# Read the Excel file
try:
    df = pd.read_excel(input_file)
except Exception as e:
    raise Exception(f"Error loading '{input_file}': {str(e)}")

# Standardize column names (handle case sensitivity, spaces, etc.)
df.columns = df.columns.str.strip().str.lower()

# Check for required columns
required_columns = ['date', 'open', 'close', 'high', 'low']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}. Found columns: {list(df.columns)}")

# Function to convert Excel serial date or Timestamp to YYYY-MM-DD
def convert_date(date):
    if isinstance(date, (int, float)):  # Excel serial date
        base_date = datetime(1899, 12, 30)  # Excel's base date
        return (base_date + timedelta(days=date)).strftime('%Y-%m-%d')
    elif isinstance(date, pd.Timestamp):  # Already a Timestamp
        return date.strftime('%Y-%m-%d')
    else:  # Fallback for unexpected types
        return str(date)

# 1. Convert Date column to YYYY-MM-DD
df['date'] = df['date'].apply(convert_date)

# 2. Round price columns to 2 decimal places
price_columns = ['close', 'high', 'low', 'open']
for col in price_columns:
    df[col] = df[col].round(2)

# 3. Flag potential outliers (price changes > 20% day-to-day)
for i in range(1, len(df)):
    prev_close = df.loc[i-1, 'close']
    curr_close = df.loc[i, 'close']
    if prev_close != 0:
        pct_change = abs((curr_close - prev_close) / prev_close * 100)
        if pct_change > 20:
            logging.info(f"Potential outlier on {df.loc[i, 'date']}: Close price changed from {prev_close} to {curr_close} ({pct_change:.2f}%)")

# 4. Remove rows with zero volume (if Volume column exists)
if 'volume' in df.columns:
    df = df[df['volume'] != 0]
    # 5. Drop the Volume column
    df = df.drop(columns=['volume'])
else:
    print("Warning: 'volume' column not found, skipping zero-volume removal and drop.")

# 6. Exclude the last row if it's incomplete (check for NaN in required columns)
if df[required_columns].iloc[-1].isna().any():
    df = df.iloc[:-1]

# 7. Ensure consistent data types (floats for prices)
for col in price_columns:
    df[col] = df[col].astype(float)

# 8. Capitalize column names for output (Date, Open, Close, High, Low)
df = df.rename(columns={'date': 'Date', 'open': 'Open', 'close': 'Close', 'high': 'High', 'low': 'Low'})

# 9. Save to a clean CSV file
try:
    df.to_csv(output_file, index=False)
    print(f"Cleaned data saved to '{output_file}'")
except Exception as e:
    raise Exception(f"Error saving to '{output_file}': {str(e)}")

Cleaned data saved to 'training\csv\hindpetro900d.csv'
